# Instalo e importo librerías

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=e8c9a267ae1fdd07026e96c5153d3c4a34cf1ed8e8945fa65e584d946f0a753b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected packag

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Google Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Creo sesión de spark y leo csvs

In [ ]:
downloaded = drive.CreateFile({'id':"1gDD0Mn9bYoj4RQlPb7V2_3mJ_-Ikmit8"})
downloaded.GetContentFile('GooglePlayStore.csv')

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('GooglePlayStore.csv', header=True, inferSchema=True)
apps_details = df.rdd

# Ejercicio 16

Ordenar de forma descendente todas las categorías por cantidad de descargas. (⭐⭐)

Consideraré cantidad de descargas de una app como el valor que aparece en 'Installs'. Este tiene formato numérico, en algunos casos con un '+' al final, por lo que realmente puedo trabajar con cantidades aproximadas de instalaciones/descargas, no cuento con el dato exacto. Lo que haré es eliminar el '+' y quedarme con el número que haya, por más que sea una cota mínima de la cantidad real de descargas.


Hay una app que figura con cantidad de instalaciones sin sentido, como "Free" (fácil de hallar porque tiene todos los datos mal cargados, en especial tiene rating 19, un absurdo). La filtraré inicialmente.

Mapeo: me quedo con app como clave y como valor: (Categoría, #Instalaciones).

Hay apps que figuran repetidas. Estas no las tendré en cuenta ya que puedo tener valores que no coinciden en la cantidad de instalaciones, o en el caso de estar repetidos no aportan nada (consideraré como valor válido de instalaciones el primero de los repetidos que aparece en mis datos). No solo por las instalaciones, sino que a a veces hay discrepancias entra las categorías de una app, me quedaré con la primera que aparezca para una aplicación (con el reduceByKey).

Luego, vuelvo a mapear para quedarme con estructura clave-valor pero esta vez la categoría como clave y la cantidad de instalaciones como valor. Hago un reduceByKey sumando, para obtener la cantidad de instalaciones por categoría.

Cacheo todo lo anterior, ya que luego haré dos operaciones sobre este rdd para mostrar todas las categorías por cantidad de descargas/instalaciones (un takeOrdered y un count)

In [ ]:
downloads_by_category = apps_details.filter(lambda x: x.Installs != 'Free').map(lambda x: (x.App, (x.Category, int(x.Installs.replace('+','').replace(',','')))))\
.reduceByKey(lambda a,b: a).map(lambda x: x[1]).reduceByKey(lambda x,y: x+y).cache()

In [ ]:
downloads_by_category.takeOrdered(downloads_by_category.count(), lambda x: -x[1])

[('GAME', 13878924415),
 ('COMMUNICATION', 11038276251),
 ('TOOLS', 8001771915),
 ('PRODUCTIVITY', 5793091369),
 ('SOCIAL', 5487867902),
 ('PHOTOGRAPHY', 4649147655),
 ('FAMILY', 4427941505),
 ('VIDEO_PLAYERS', 3926902720),
 ('TRAVEL_AND_LOCAL', 2894887146),
 ('NEWS_AND_MAGAZINES', 2369217760),
 ('ENTERTAINMENT', 2113660000),
 ('BOOKS_AND_REFERENCE', 1665969576),
 ('PERSONALIZATION', 1532494782),
 ('SHOPPING', 1400348785),
 ('HEALTH_AND_FITNESS', 1144022512),
 ('SPORTS', 1096474498),
 ('BUSINESS', 697164865),
 ('LIFESTYLE', 503823539),
 ('MAPS_AND_NAVIGATION', 503281890),
 ('FINANCE', 455348734),
 ('WEATHER', 361100520),
 ('EDUCATION', 352952000),
 ('FOOD_AND_DRINK', 211798751),
 ('DATING', 140926107),
 ('ART_AND_DESIGN', 114338100),
 ('HOUSE_AND_HOME', 97212461),
 ('AUTO_AND_VEHICLES', 53130211),
 ('LIBRARIES_AND_DEMO', 52995910),
 ('COMICS', 44981150),
 ('MEDICAL', 38193177),
 ('PARENTING', 31521110),
 ('BEAUTY', 27197050),
 ('EVENTS', 15973161)]